In [ ]:
# ref： https://cooltools.readthedocs.io/en/latest/notebooks/command_line_interface.html
# ref： https://cooltools.readthedocs.io/en/latest/notebooks/insulation_and_boundaries.html
import os 
print(os.getcwd())
os.chdir("/data1/sunwenju/projects_on_940xa/2024041703_mESC_HiC_use_pairtools_cooler_pipeline/Nora_IAA")
print(os.getcwd())

In [ ]:
! ls -1 *.mcool

In [ ]:
nora_iaa_rep12_mcool = "Nora_mESC_IAA_HiC.rep12.merged.1k.zoomify.mcool"
nora_iaa_rep1_mcool = "Nora_mESC_IAA_HiC_rep1.nodups.pairs.1k.zoomify.mcool"
nora_iaa_rep2_mcool = "Nora_mESC_IAA_HiC_rep2.nodups.pairs.1k.zoomify.mcool"

In [ ]:
import cooler
# to print which resolutions are stored in the mcool, use list_coolers
cooler.fileops.list_coolers(nora_iaa_rep12_mcool)

In [ ]:
import pandas as pd
mm10_view_df = pd.read_csv('../mm10.chrom.view.tsv', sep='\t', header=0)
mm10_view_df.head()

# nora_iaa_rep12_mcool

## compute r20k insulation_table

In [ ]:
import pandas as pd
import cooler
from cooltools import insulation
resolution=20_000
mm10_view_df = pd.read_csv("../mm10.chrom.view.tsv", sep="\t", header=0)
nora_iaa_rep12_mcool = "Nora_mESC_IAA_HiC.rep12.merged.1k.zoomify.mcool"
nora_iaa_rep12_mcool_r20k = cooler.Cooler(f'{nora_iaa_rep12_mcool}::resolutions/{resolution}')
windows = [5*resolution, 10*resolution, 15*resolution]
insulation_table = insulation(clr=nora_iaa_rep12_mcool_r20k,
                              window_bp=windows, view_df=mm10_view_df,
                              verbose=True, nproc=64)
insulation_table.to_csv("Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.insulation_table.tsv", sep="\t", index=False, header=True, na_rep="NaN")
print(insulation_table.shape)
insulation_table.head()


In [ ]:
r20k_badbin_f, r20k_badbin_t = insulation_table.is_bad_bin.value_counts()
r20k_w5r_f, r20k_w5r_t = insulation_table[f'is_boundary_{5*resolution}'].value_counts()
r20k_w10r_f, r20k_w10r_t = insulation_table[f'is_boundary_{10*resolution}'].value_counts()
r20k_w15r_f, r20k_w15r_t = insulation_table[f'is_boundary_{15*resolution}'].value_counts()
print(r20k_badbin_t, r20k_w5r_t, r20k_w10r_t, r20k_w15r_t, sep="\t")
print(r20k_badbin_f, r20k_w5r_f, r20k_w10r_f, r20k_w15r_f, sep="\t")

In [ ]:
r20k_w5r_TAD_boundary_df = insulation_table[insulation_table[f'is_boundary_{5*resolution}'] == True][["chrom", "start", "end"]]
r20k_w5r_TAD_boundary_df.to_csv("Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.w5r_insulation_boundary.bed", sep="\t", index=False, header=False)
print(r20k_w5r_TAD_boundary_df.shape)
r20k_w5r_TAD_boundary_df.head()


In [ ]:
r20k_w10r_TAD_boundary_df = insulation_table[insulation_table[f'is_boundary_{10*resolution}'] == True][["chrom", "start", "end"]]
r20k_w10r_TAD_boundary_df.to_csv("Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.w10r_insulation_boundary.bed", sep="\t", index=False, header=False)
print(r20k_w10r_TAD_boundary_df.shape)
r20k_w10r_TAD_boundary_df.head()

In [ ]:
r20k_w15r_TAD_boundary_df = insulation_table[insulation_table[f'is_boundary_{15*resolution}'] == True][["chrom", "start", "end"]]
r20k_w15r_TAD_boundary_df.to_csv("Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.w15r_insulation_boundary.bed", sep="\t", index=False, header=False)
print(r20k_w15r_TAD_boundary_df.shape)
r20k_w15r_TAD_boundary_df.head()


## compute r20k w5r/w10r/w15r TADs and stat

In [ ]:
import bioframe
def extract_TADs(insulation_table, is_boundary_col, max_TAD_length = 3_000_000):
    tads = bioframe.merge(insulation_table[insulation_table[is_boundary_col] == False])
    return tads[(tads["end"] - tads["start"]) <= max_TAD_length].reset_index(drop=True)[['chrom', 'start', 'end']]

r20k_w5r_TADS = extract_TADs(insulation_table, f'is_boundary_{resolution*5}')
r20k_w10r_TADS = extract_TADs(insulation_table, f'is_boundary_{resolution*10}')
r20k_w15r_TADS = extract_TADs(insulation_table, f'is_boundary_{resolution*15}')
r20k_w5r_TADS.to_csv("Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.w5r.TADs.bed", sep="\t", index=False, header=False)
r20k_w10r_TADS.to_csv("Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.w10r.TADs.bed", sep="\t", index=False, header=False)
r20k_w15r_TADS.to_csv("Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.w15r.TADs.bed", sep="\t", index=False, header=False)
print(r20k_w5r_TADS.shape[0], r20k_w10r_TADS.shape[0], r20k_w15r_TADS.shape[0], sep="\t")


## TADs view plot

In [ ]:
import bioframe
# Visualization imports:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.ticker import EngFormatter
# helper functions for plotting
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    """format ticks with genomic coordinates as human readable"""
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

# create a functions that would return a series of rectangles around called tads
# in a specific region, and exposing importnat plotting parameters
def rectangles_around_tads(tads_df, plot_region, loc="none", lw=1, ec="lime", fc="none"):
    """
    yield a series of rectangles around called tads in a given region
    """
    # select tads from the region:
    df_reg = bioframe.select(tads_df, plot_region, cols=("chrom", "start", "end"))
    Polygon_kwargs = dict(lw=lw, edgecolor=ec, facecolor=fc)
    # draw rectangular "boxes" around pixels called as tads in the "region":
    for tad_s, tad_e in df_reg[["start", "end"]].itertuples(index=False):
        if loc == "upper":
            # 注意：多边形顶点顺序需要按照逆时针或顺时针排列
            yield patches.Polygon([(tad_s, tad_s), (tad_e, tad_e), (tad_e, tad_s)], closed=True, **Polygon_kwargs)
        elif loc == "lower":
            yield patches.Polygon([(tad_s, tad_s), (tad_s, tad_e), (tad_e, tad_e)], closed=True, **Polygon_kwargs)
        else:
            yield patches.Polygon([(tad_s, tad_s), (tad_s, tad_e), (tad_e, tad_e), (tad_e, tad_s)], closed=True, **Polygon_kwargs)


### Fig2A TADs view

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import fill_diag
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# define a region to look into as an example
chromsome = "chr1"
start = 16_000_000
end = 18_000_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

# # # compute heatmap for the region
# # region_matrix = nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(region)
# for diag in [-2, -1, 0, 1, 2]:
#     nora_iaa_rep12_mcool_r20k_adcg_interp = fill_diag(nora_iaa_rep12_mcool_r20k_adcg_interp, np.nan, i=diag)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(nora_iaa_rep12_mcool_r20k_adcg*100000, vmin=0, vmax=1300, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'Nora_IAA_r20k Fig2A ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

# draw rectangular "boxes" around pixels called as tads in the "region":
for box in rectangles_around_tads(r20k_w5r_TADS, plot_region, lw=1.5, loc="none"):
    ax1.add_patch(box)

plt.savefig("Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.view.Fig2A.fall.heatmap.with_TADs.pdf")
plt.savefig("Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.view.Fig2A.fall.heatmap.with_TADs.png")


### Fig2B TADs view

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import fill_diag
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# define a region to look into as an example
chromsome = "chr9"
start = 13_800_000
end = 15_500_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

# # # compute heatmap for the region
# # region_matrix = nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(region)
# for diag in [-2, -1, 0, 1, 2]:
#     nora_iaa_rep12_mcool_r20k_adcg_interp = fill_diag(nora_iaa_rep12_mcool_r20k_adcg_interp, np.nan, i=diag)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(nora_iaa_rep12_mcool_r20k_adcg*100000, vmin=0, vmax=1300, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1200, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'Nora_IAA_r20k Fig2B ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

# draw rectangular "boxes" around pixels called as tads in the "region":
for box in rectangles_around_tads(r20k_w5r_TADS, plot_region, lw=1.5, loc="none"):
    ax1.add_patch(box)

plt.savefig("Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.view.Fig2B.fall.heatmap.with_TADs.pdf")
plt.savefig("Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.view.Fig2B.fall.heatmap.with_TADs.png")


### Fig3E TADs view

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import fill_diag
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# define a region to look into as an example
chromsome = "chr1"
start = 9_800_000
end = 14_600_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

# # # compute heatmap for the region
# # region_matrix = nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(region)
# for diag in [-2, -1, 0, 1, 2]:
#     nora_iaa_rep12_mcool_r20k_adcg_interp = fill_diag(nora_iaa_rep12_mcool_r20k_adcg_interp, np.nan, i=diag)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(nora_iaa_rep12_mcool_r20k_adcg*100000, vmin=0, vmax=1300, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'Nora_IAA_r20k Fig3E ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

# draw rectangular "boxes" around pixels called as tads in the "region":
for box in rectangles_around_tads(r20k_w5r_TADS, plot_region, lw=1.5, loc="none"):
    ax1.add_patch(box)

plt.savefig("Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.view.Fig3E.fall.heatmap.with_TADs.pdf")
plt.savefig("Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.view.Fig3E.fall.heatmap.with_TADs.png")


### FigS3J TADs view

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import fill_diag
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# define a region to look into as an example
chromsome = "chr14"
start = 109_400_000
end = 111_900_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

# # # compute heatmap for the region
# # region_matrix = nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(region)
# for diag in [-2, -1, 0, 1, 2]:
#     nora_iaa_rep12_mcool_r20k_adcg_interp = fill_diag(nora_iaa_rep12_mcool_r20k_adcg_interp, np.nan, i=diag)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(nora_iaa_rep12_mcool_r20k_adcg*100000, vmin=0, vmax=1300, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'Nora_IAA_r20k FigS3J ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

# draw rectangular "boxes" around pixels called as tads in the "region":
for box in rectangles_around_tads(r20k_w5r_TADS, plot_region, lw=1.5, loc="none"):
    ax1.add_patch(box)

plt.savefig("Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.view.FigS3J.fall.heatmap.with_TADs.pdf")
plt.savefig("Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.view.FigS3J.fall.heatmap.with_TADs.png")


### Malat1 TADs view

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import fill_diag
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# define a region to look into as an example
chromsome = "chr19"
start = 5_000_000
end = 6_500_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

# # # compute heatmap for the region
# # region_matrix = nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(region)
# for diag in [-2, -1, 0, 1, 2]:
#     nora_iaa_rep12_mcool_r20k_adcg_interp = fill_diag(nora_iaa_rep12_mcool_r20k_adcg_interp, np.nan, i=diag)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(nora_iaa_rep12_mcool_r20k_adcg*100000, vmin=0, vmax=1300, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1500, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'Nora_IAA_r20k Malat1 ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

# draw rectangular "boxes" around pixels called as tads in the "region":
for box in rectangles_around_tads(r20k_w5r_TADS, plot_region, lw=1.5, loc="none"):
    ax1.add_patch(box)

plt.savefig("Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.view.Malat1.fall.heatmap.with_TADs.pdf")
plt.savefig("Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.view.Malat1.fall.heatmap.with_TADs.png")


### Myc TADs view

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import fill_diag
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# define a region to look into as an example
chromsome = "chr15"
start = 61_000_000
end = 63_000_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

# # # compute heatmap for the region
# # region_matrix = nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(region)
# for diag in [-2, -1, 0, 1, 2]:
#     nora_iaa_rep12_mcool_r20k_adcg_interp = fill_diag(nora_iaa_rep12_mcool_r20k_adcg_interp, np.nan, i=diag)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(nora_iaa_rep12_mcool_r20k_adcg*100000, vmin=0, vmax=1300, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'Nora_IAA_r20k Myc ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

# draw rectangular "boxes" around pixels called as tads in the "region":
for box in rectangles_around_tads(r20k_w5r_TADS, plot_region, lw=1.5, loc="none"):
    ax1.add_patch(box)

plt.savefig("Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.view.Myc.fall.heatmap.with_TADs.pdf")
plt.savefig("Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.view.Myc.fall.heatmap.with_TADs.png")


### Nanog TADs view

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import fill_diag
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# define a region to look into as an example
chromsome = "chr6"
start = 121_500_000
end = 123_500_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

# # # compute heatmap for the region
# # region_matrix = nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(region)
# for diag in [-2, -1, 0, 1, 2]:
#     nora_iaa_rep12_mcool_r20k_adcg_interp = fill_diag(nora_iaa_rep12_mcool_r20k_adcg_interp, np.nan, i=diag)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(nora_iaa_rep12_mcool_r20k_adcg*100000, vmin=0, vmax=1300, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1200, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'Nora_IAA_r20k Nanog ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

# draw rectangular "boxes" around pixels called as tads in the "region":
for box in rectangles_around_tads(r20k_w5r_TADS, plot_region, lw=1.5, loc="none"):
    ax1.add_patch(box)

plt.savefig("Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.view.Nanog.fall.heatmap.with_TADs.pdf")
plt.savefig("Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.view.Nanog.fall.heatmap.with_TADs.png")


### Pax6 TADs view

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import fill_diag
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# define a region to look into as an example
chromsome = "chr2"
start = 104_500_000
end = 106_500_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

# # # compute heatmap for the region
# # region_matrix = nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(region)
# for diag in [-2, -1, 0, 1, 2]:
#     nora_iaa_rep12_mcool_r20k_adcg_interp = fill_diag(nora_iaa_rep12_mcool_r20k_adcg_interp, np.nan, i=diag)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(nora_iaa_rep12_mcool_r20k_adcg*100000, vmin=0, vmax=1300, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'Nora_IAA_r20k Pax6 ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

# draw rectangular "boxes" around pixels called as tads in the "region":
for box in rectangles_around_tads(r20k_w5r_TADS, plot_region, lw=1.5, loc="none"):
    ax1.add_patch(box)

plt.savefig("Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.view.Pax6.fall.heatmap.with_TADs.pdf")
plt.savefig("Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.view.Pax6.fall.heatmap.with_TADs.png")


### Pou5f1 TADs view

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import fill_diag
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# define a region to look into as an example
chromsome = "chr17"
start = 34_500_000
end = 36_500_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

# # # compute heatmap for the region
# # region_matrix = nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(region)
# for diag in [-2, -1, 0, 1, 2]:
#     nora_iaa_rep12_mcool_r20k_adcg_interp = fill_diag(nora_iaa_rep12_mcool_r20k_adcg_interp, np.nan, i=diag)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(nora_iaa_rep12_mcool_r20k_adcg*100000, vmin=0, vmax=1300, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1600, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'Nora_IAA_r20k Pou5f1 ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

# draw rectangular "boxes" around pixels called as tads in the "region":
for box in rectangles_around_tads(r20k_w5r_TADS, plot_region, lw=1.5, loc="none"):
    ax1.add_patch(box)

plt.savefig("Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.view.Pou5f1.fall.heatmap.with_TADs.pdf")
plt.savefig("Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.view.Pou5f1.fall.heatmap.with_TADs.png")


### Sox1 TADs view

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import fill_diag
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# define a region to look into as an example
chromsome = "chr8"
start = 11_000_000
end = 13_000_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

# # # compute heatmap for the region
# # region_matrix = nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(region)
# for diag in [-2, -1, 0, 1, 2]:
#     nora_iaa_rep12_mcool_r20k_adcg_interp = fill_diag(nora_iaa_rep12_mcool_r20k_adcg_interp, np.nan, i=diag)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(nora_iaa_rep12_mcool_r20k_adcg*100000, vmin=0, vmax=1300, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1100, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'Nora_IAA_r20k Sox1 ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

# draw rectangular "boxes" around pixels called as tads in the "region":
for box in rectangles_around_tads(r20k_w5r_TADS, plot_region, lw=1.5, loc="none"):
    ax1.add_patch(box)

plt.savefig("Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.view.Sox1.fall.heatmap.with_TADs.pdf")
plt.savefig("Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.view.Sox1.fall.heatmap.with_TADs.png")


### Sox2 TADs view

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import fill_diag
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# define a region to look into as an example
chromsome = "chr3"
start = 33_500_000
end = 35_500_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

# # # compute heatmap for the region
# # region_matrix = nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(region)
# for diag in [-2, -1, 0, 1, 2]:
#     nora_iaa_rep12_mcool_r20k_adcg_interp = fill_diag(nora_iaa_rep12_mcool_r20k_adcg_interp, np.nan, i=diag)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(nora_iaa_rep12_mcool_r20k_adcg*100000, vmin=0, vmax=1300, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1000, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'Nora_IAA_r20k Sox2 ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

# draw rectangular "boxes" around pixels called as tads in the "region":
for box in rectangles_around_tads(r20k_w5r_TADS, plot_region, lw=1.5, loc="none"):
    ax1.add_patch(box)

plt.savefig("Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.view.Sox2.fall.heatmap.with_TADs.pdf")
plt.savefig("Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.view.Sox2.fall.heatmap.with_TADs.png")


### Zic1 TADs view

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import fill_diag
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# define a region to look into as an example
chromsome = "chr9"
start = 89_000_000
end = 93_000_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

# # # compute heatmap for the region
# # region_matrix = nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(region)
# for diag in [-2, -1, 0, 1, 2]:
#     nora_iaa_rep12_mcool_r20k_adcg_interp = fill_diag(nora_iaa_rep12_mcool_r20k_adcg_interp, np.nan, i=diag)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(nora_iaa_rep12_mcool_r20k_adcg*100000, vmin=0, vmax=1300, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'Nora_IAA_r20k Zic1 ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

# draw rectangular "boxes" around pixels called as tads in the "region":
for box in rectangles_around_tads(r20k_w5r_TADS, plot_region, lw=1.5, loc="none"):
    ax1.add_patch(box)

plt.savefig("Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.view.Zic1.fall.heatmap.with_TADs.pdf")
plt.savefig("Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.view.Zic1.fall.heatmap.with_TADs.png")
